# Exámen parcial Bourbaki track

En [este sitio](https://developer.ibm.com/exchanges/data/all/project-codenet/) considere el dataset llamado Project_CodeNet_LangClass.tgz el cual consiste en un conjunto de códigos en 10 lenguajes de programación distintos. Además es posible construir una etiqueta que corresponda al lenguaje de programación al que pertenece.

Se deberá de construir una red neuronal para clasificar estos códigos de acuerdo al lenguaje de programación correspondiente. Idealmente sugerimos utilizar una red neuronal recurrente aunque es posible utilizar otros acercamientos. Las funciones de activación y de pérdida sugeridas son Softmax y Cross-Entropy respectivamente.

Es importante notar que el proceso de tokenización no es de ninguna manera obvio, se podría trabajar con una bolsa de palabras hecha
únicamente con caracteres.

También se podría entrenar un encaje al estilo de los autoencoders que estudiamos, en el sitio donde se puede descargar el dataset es posible descargar un conjunto de datos más grande con el que se podría entrenar un
tal encaje. Hacer esto no es indispensable pues es más complicado. 

Pueden trabajar en local o en Google Colab, al finalizar deberán enviar el archivo o el link con permisos para comentar a Braulio, Édison Vázquez y Alfonso Ruiz.

In [3]:
import pandas as pd
import numpy as np

# Preprocesamiento

Primero voy a leer cada archivo línea por línea. Para eso definiré la siguiente función read_code_files, la cual genera una lista de tuplets con el código como texto y el típo de archivo (c, py, etc.). Nótese que la lectura del texto respeta el salto de línea.

In [4]:
from pathlib import Path

def read_code_files(folder_path):
    file_paths = list(Path(folder_path).rglob("*.*"))
    data = []
    for path in file_paths:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()
            label = path.suffix[1:]  # 'py', 'c', etc.
            data.append((content, label))
    return data

In [30]:
df = read_code_files("data/")

Después convierto esta lista en un dataframe de pandas.

In [31]:
df = pd.DataFrame(df, columns=["code", "label"])
df.sample(10)

,code,label
590,import Control.Monad\nimport Control.Applicati...,hs
534,\n//Digit Number\nimport java.io.BufferedReade...,java
211,###\n### atcorder test program\n###\n\nimport ...,py
29,//using System;\n//class p0003\n//{\n// sta...,cs
513,public class Main {\n\n public static void ...,java
353,using System;\nusing System.IO;\nusing System....,cs
103,#include <bits/stdc++.h>\nusing namespace std;...,cpp
523,import java.io.*;\n\nclass Main {\n\tpublic st...,java
656,<?php\n\nwhile (($line = trim(fgets(STDIN))) !...,php
214,# coding=utf-8\n\n\ndef direction_vector(p1: l...,py


## Tokenización del texto

En este ejercicio voy a tokenizar por palabras o símbolos individuales. 

In [32]:
import re

def tokenize(code):
    return re.findall(r'\w+|[^\s\w]', code)

In [33]:
tokenize(df["code"][0])

['\x00',
 '\x00',
 '\x00',
 '\x01',
 'Bud1',
 '\x00',
 '\x00',
 '\x10',
 '\x00',
 '\x00',
 '\x00',
 '\x08',
 '\x00',
 '\x00',
 '\x00',
 '\x10',
 '\x00',
 '\x00',
 '\x00',
 '\x01',
 '\x08',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x08',
 '\x00',
 '\x00',
 '\x00',
 '\x08',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x02',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x02',
 '\x00',
 '\x00',
 '\x00',
 '\x01',
 '\x00',
 '\x00',
 '\x10',
 '\x00',
 'bwspblob',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00

Construimos el vocabulario

In [34]:
from collections import Counter

# Tokenizamos todos los códigos
all_tokens = []
for code in df['code']:
    tokens = tokenize(code)
    all_tokens.extend(tokens)

# Construimos vocabulario
token_freqs = Counter(all_tokens)
vocab = {token: i+2 for i, (token, _) in enumerate(token_freqs.items())}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1

Mapeamos tokens a índices

In [35]:
def tokens_to_ids(tokens, vocab):
    return [vocab.get(t, vocab["<UNK>"]) for t in tokens]

Preparamos el tensor con el que entrenaremos nuestra RNN.

Primero codificamos nuestras etiquetas con LabelEncoder, después tokenizamos nuestos textos y lo convertimos a índices, después hacemos padding a las secuencias, convertimos estas secuencias a un tensor y finalmente convertimos a tensor estas secuencias.

In [41]:
from sklearn.preprocessing import LabelEncoder
import torch
from torch.nn.utils.rnn import pad_sequence

# Codificamos etiquetas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df["label"])

# Secuencias tokenizadas
MAX_LEN = 500
encoded_sequences = []

for code in df["code"]:
    tokens = tokenize(code)
    token_ids = tokens_to_ids(tokens[:MAX_LEN], vocab)
    encoded_sequences.append(torch.tensor(token_ids, dtype=torch.long))

# Padding
padded_sequences = pad_sequence(encoded_sequences, batch_first=True, padding_value=vocab["<PAD>"])
labels_tensor = torch.tensor(y_encoded, dtype=torch.long)

print(padded_sequences.shape)  # (num_samples, MAX_LEN)
print(labels_tensor.shape)     # (num_samples,)

torch.Size([1002, 500])
torch.Size([1002])


In [26]:
padded_sequences

tensor([[   2,    2,    3,  ...,  105,  113,   17],
        [   5,    6,    7,  ...,   13,  231,   17],
        [   5,    6,    7,  ...,   13,  231,   17],
        ...,
        [ 709,   16,   11,  ..., 6300,   17, 6539],
        [ 709,   16,   11,  ...,   40,  393,   99],
        [ 709,   16,   11,  ...,    0,    0,    0]])

Ahora crearemos el dataset con el que entrenaremos nuestra RNN. Primero crearemos un tensor combinando nuestras secuencias y las etiquetas correspondientes. Después lo dividiremos en conjuntos de entrenamiento y validación.

In [43]:
from torch.utils.data import TensorDataset, DataLoader, random_split

dataset = TensorDataset(padded_sequences, labels_tensor)

# División train/val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

## Diseño del modelo

In [44]:
import torch.nn as nn

class CodeClassifierRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, embed_dim)
        _, (hn, _) = self.lstm(x)  # hn: (1, batch, hidden_dim)
        return self.fc(hn.squeeze(0))

## Red neuronal recurrente

In [47]:
model = CodeClassifierRNN(
    vocab_size=len(vocab),
    embed_dim=128,
    hidden_dim=256,
    num_classes=len(label_encoder.classes_)
)

# Evaluación del modelo

In [48]:
def evaluate_model(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs = inputs.to(device)

            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    acc = accuracy_score(all_labels, all_preds)
    return acc

In [55]:
import torch
from torch import nn, optim
from sklearn.metrics import classification_report, accuracy_score

model = CodeClassifierRNN(
    vocab_size=len(vocab),
    embed_dim=128,
    hidden_dim=256,
    num_classes=len(label_encoder.classes_)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [56]:
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        val_acc = evaluate_model(model, val_loader)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_loss:.4f} | Val Accuracy: {val_acc:.4f}")

In [57]:
def final_report(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

In [58]:
train_model(model, train_loader, val_loader, epochs=100)
final_report(model, val_loader)

Epoch 1/100 | Train Loss: 2.1226 | Val Accuracy: 0.3632
Epoch 2/100 | Train Loss: 1.8011 | Val Accuracy: 0.4129
Epoch 3/100 | Train Loss: 1.9958 | Val Accuracy: 0.3284
Epoch 4/100 | Train Loss: 1.7761 | Val Accuracy: 0.4129
Epoch 5/100 | Train Loss: 1.6959 | Val Accuracy: 0.4080
Epoch 6/100 | Train Loss: 1.4280 | Val Accuracy: 0.4677
Epoch 7/100 | Train Loss: 1.4065 | Val Accuracy: 0.3980
Epoch 8/100 | Train Loss: 1.4851 | Val Accuracy: 0.4478
Epoch 9/100 | Train Loss: 1.3520 | Val Accuracy: 0.4876
Epoch 10/100 | Train Loss: 1.5026 | Val Accuracy: 0.4726
Epoch 11/100 | Train Loss: 1.5956 | Val Accuracy: 0.4677
Epoch 12/100 | Train Loss: 1.1153 | Val Accuracy: 0.5672
Epoch 13/100 | Train Loss: 0.9785 | Val Accuracy: 0.5672
Epoch 14/100 | Train Loss: 1.0507 | Val Accuracy: 0.5323
Epoch 15/100 | Train Loss: 1.0320 | Val Accuracy: 0.5473
Epoch 16/100 | Train Loss: 1.3689 | Val Accuracy: 0.5075
Epoch 17/100 | Train Loss: 1.4811 | Val Accuracy: 0.4975
Epoch 18/100 | Train Loss: 1.1564 | Val 